In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation

# Loading In Data

In [ ]:
with np.load('data/train_and_test.npz') as data:
    X_test = data['X_test']
    y_train = data['y_train']
    X_train = data['X_train']

In [ ]:
plt.imshow(X_train[20000])
plt.title(y_train[20000])

# Keras Convolutional Neural Network

In [ ]:
INPUT_SHAPE = (32, 32, 3)
KERNEL_SIZE = (2, 2)
POOL_SIZE = (2, 2)
STRIDES = 2
DROP_RATE = 0.25
LEARNING_RATE = 0.001

model = Sequential([
    Conv2D(32, kernel_size=KERNEL_SIZE, padding='same', activation='relu', input_shape=INPUT_SHAPE),
    BatchNormalization(),
    MaxPooling2D(pool_size=POOL_SIZE, strides=STRIDES),
    Dropout(DROP_RATE),
    Conv2D(64, kernel_size=KERNEL_SIZE, activation='relu', border_mode='same'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(DROP_RATE),
    Conv2D(64, kernel_size=KERNEL_SIZE, activation='relu', border_mode='same'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(DROP_RATE),
    Flatten(),
    Dense(64, use_bias=False),
    BatchNormalization(),
    Activation('relu'),
    Dropout(DROP_RATE),
    Dense(43),
    Activation('softmax'),
])

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=['accuracy'],
)

model.summary()

# Training Model

In [ ]:
y_categorical = to_categorical(y_train)
model.fit(X_train, y_categorical, epochs=20, batch_size=32)

# Predict And Produce Submission File

In [ ]:
result = [list(prediction).index(max(prediction)) for prediction in model.predict(X_test)]

with open('submission.csv', 'w') as submission_file:
    submission_file.write('id, labels\n')
    for index, label in enumerate(result):
        submission_file.write('{},{}\n'.format(index, label))